# Start

In [3]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

# Links:

[[cameras_st_asgard]]
[[cameras_dir_partner]]
[[companies_st_partner]]
[[companies_dir_partner]]

# Create table


In [12]:
query_text = """--sql
    CREATE TABLE db1.t_cameras_by_companies 
    (
        `tariff_full` String,
        `report_date` Date, 
        `partner_uuid` String,
        `partner_lk` String,
        `company_name` String,
        `tin` String,
        `kpp` String,
        `camera_with_intercom_count` UInt32,
        `external_camera_izi_count` UInt32,
        `external_camera_foreign_count` UInt32
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """
ch.query_run(query_text)

""


# Create mv

In [ ]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_cameras_by_companies_mv
REFRESH EVERY 1 DAY OFFSET 5 HOUR 15 MINUTE TO db1.t_cameras_by_companies AS
SELECT
    comp_s_par.tariff_full AS tariff_full,
    cam_s_asg.report_date AS report_date, 
    cam_s_asg.partner_uuid AS partner_uuid,
    comp_d_par.partner_lk AS partner_lk,
    comp_d_par.company_name AS company_name,
    comp_d_par.tin AS tin,
    comp_d_par.kpp AS kpp,
    SUM(IF(camera_with_intercom = 1, 1,0)) AS camera_with_intercom_count,
    SUM(IF(camera_with_intercom = 0 AND COALESCE(foreign_camera,0) = 0,1,0)) AS external_camera_izi_count,
    SUM(IF(camera_with_intercom = 0 AND foreign_camera = 1,1,0)) AS external_camera_foreign_count
FROM
    db1.cameras_st_asgard_ch AS cam_s_asg
    LEFT JOIN db1.cameras_dir_partner_ch AS cam_d_par ON cam_s_asg.camera_uuid = cam_d_par.camera_uuid
    LEFT JOIN db1.companies_st_partner_ch AS comp_s_par 
    						ON cam_s_asg.partner_uuid = comp_s_par.partner_uuid
                            AND cam_s_asg.report_date = comp_s_par.report_date
    LEFT JOIN db1.companies_dir_partner_ch AS comp_d_par 
    						ON comp_s_par.partner_uuid = comp_d_par.partner_uuid
GROUP BY 
    comp_s_par.tariff_full AS tariff_full,
    cam_s_asg.report_date AS report_date, 
    cam_s_asg.partner_uuid AS partner_uuid,
    comp_d_par.partner_lk AS partner_lk,
    comp_d_par.company_name AS company_name,
    comp_d_par.tin AS tin,
    comp_d_par.kpp AS kpp
    """
ch.query_run(query_text)

""


# Additional

In [ ]:
query_text = """--sql
SELECT
    DISTINCT
    partner_uuid,
    
FROM db1.billing_orders_dir_partner
limit 10
    """
ch.query_run(query_text)

,created_at,state,service,cost,total,count,billing_account_id,partner_uuid
0,2018-11-13 06:55:13,success,zevs,50.0,2050.0,0,3,b1782e4f-9198-49d1-b5aa-7bdba9c87d21
1,2018-11-13 07:51:38,success,zevs,50.0,2050.0,0,3,b1782e4f-9198-49d1-b5aa-7bdba9c87d21
2,2018-11-13 07:51:38,success,glashatai,50.0,2050.0,0,3,b1782e4f-9198-49d1-b5aa-7bdba9c87d21
3,2018-11-14 14:28:35,success,zevs,50.0,2050.0,0,3,b1782e4f-9198-49d1-b5aa-7bdba9c87d21
4,2018-11-14 14:28:35,success,glashatai,50.0,2050.0,0,3,b1782e4f-9198-49d1-b5aa-7bdba9c87d21
5,2018-11-14 14:28:35,success,cctv_record,200.0,200.0,0,3,b1782e4f-9198-49d1-b5aa-7bdba9c87d21
6,2018-11-14 14:28:35,success,example_service,20000.0,20000.0,0,3,b1782e4f-9198-49d1-b5aa-7bdba9c87d21
7,2018-12-28 08:51:42,success,zevs,50.0,100.0,0,6,67fcbe0d-3cea-4656-831a-2156ee67f5fc
8,2018-12-28 08:51:42,success,glashatai,50.0,2000.0,0,3,b1782e4f-9198-49d1-b5aa-7bdba9c87d21
9,2018-12-28 08:51:42,success,glashatai,50.0,100.0,0,6,67fcbe0d-3cea-4656-831a-2156ee67f5fc


In [6]:
query_text = """--sql
SELECT
    *
FROM db1.t_cameras_by_companies
LIMIT 10
    """
ch.query_run(query_text)

,tariff_full,report_date,partner_uuid,partner_lk,company_name,tin,kpp,camera_with_intercom_count,external_camera_izi_count,external_camera_foreign_count
0,,2023-07-12,,,,,,52410,4080,236
1,,2023-07-13,,,,,,52410,4081,236
2,,2023-07-14,,,,,,52408,4084,236
3,,2023-07-15,,,,,,52407,4085,236
4,,2023-07-16,,,,,,52407,4085,236
5,,2023-07-17,,,,,,52406,4086,236
6,,2023-07-18,,,,,,52406,4087,236
7,,2023-07-19,,,,,,52407,4090,236
8,,2023-07-20,,,,,,52409,4090,236
9,,2023-07-21,,,,,,52409,4092,238


In [ ]:
query_text = """
    DROP TABLE db1.companies_st_partner_mv
    """
ch.query_run(query_text)

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.t_cameras_by_companies
"""

ch.query_run(query_text)